# Deploy and score a machine learning model by using an online endpoint 

Learn how to use an online endpoint to deploy your model, so you don't have to create and manage the underlying infrastructure. You'll begin by deploying a model on your local machine to debug any errors, and then you'll deploy and test it in Azure.

Managed online endpoints help to deploy your ML models in a turnkey manner. Managed online endpoints work with powerful CPU and GPU machines in Azure in a scalable, fully managed way. Managed online endpoints take care of serving, scaling, securing, and monitoring your models, freeing you from the overhead of setting up and managing the underlying infrastructure. 

For more information, see [What are Azure Machine Learning endpoints?](https://learn.microsoft.com/azure/machine-learning/concept-endpoints), and [Deploy an ML model with an online endpoint](https://learn.microsoft.com/azure/machine-learning/how-to-deploy-online-endpoints).

## Prerequisites

* To use Azure Machine Learning, you must have an Azure subscription. If you don't have an Azure subscription, create a free account before you begin. Try the [free or paid version of Azure Machine Learning](https://azure.microsoft.com/free/).

* Install and configure the [Python SDK v2](sdk/setup.sh).

* You must have an Azure resource group, and you (or the service principal you use) must have Contributor access to it.

* You must have an Azure Machine Learning workspace. 

* To deploy locally, you must install Docker Engine on your local computer. We highly recommend this option, so it's easier to debug issues.

In [2]:
!az login

A web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "7b658efc-8cee-4580-b989-85db158d4e3c",
    "id": "3979595d-b401-4ff4-b3ef-661b8481c742",
    "isDefault": true,
    "managedByTenants": [
      {
        "tenantId": "961897a7-5808-4b80-b503-dbb7be13e7a7"
      }
    ],
    "name": "MV-AZU-001",
    "state": "Enabled",
    "tenantId": "7b658efc-8cee-4580-b989-85db158d4e3c",
    "user": {
      "name": "ben.dixon@multiverse.io",
      "type": "user"
    }
  }
]


In [3]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

/Users/ben.dixon/Repos/azure-experiments/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [4]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

Found the config file in: /Users/ben.dixon/Repos/azure-experiments/config.json


## Deploy and debug locally by using local endpoints

### Note
* To deploy locally, [Docker Engine](https://docs.docker.com/engine/install/) must be installed.
* Docker Engine must be running. Docker Engine typically starts when the computer starts. If it doesn't, you can [troubleshoot Docker Engine](https://docs.docker.com/config/daemon/#start-the-daemon-manually).

# 2. Define endpoint and deployment

## 2.1 Define the endpoint

To define an endpoint, you need to specify:

* Endpoint name: The name of the endpoint. It must be unique in the Azure region. For more information on the naming rules, see [managed online endpoint limits](how-to-manage-quotas.md#azure-machine-learning-managed-online-endpoints).
* Authentication mode: The authentication method for the endpoint. Choose between key-based authentication and Azure Machine Learning token-based authentication. A key doesn't expire, but a token does expire. For more information on authenticating, see [Authenticate to an online endpoint](how-to-authenticate-online-endpoint.md).
* Optionally, you can add a description and tags to your endpoint.

In [5]:
# Define an endpoint name
endpoint_name = "my-endpoint"

# Example way to define a random name
import datetime

endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="this is a sample online endpoint",
    auth_mode="key",
    tags={"foo": "bar"},
)

## 2.2 Define the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. To deploy a model, you must have:

- Model files (or the name and version of a model that's already registered in your workspace). In the example, we have a scikit-learn model that does regression.
- A scoring script, that is, code that executes the model on a given input request. The scoring script receives data submitted to a deployed web service and passes it to the model. The script then executes the model and returns its response to the client. The scoring script is specific to your model and must understand the data that the model expects as input and returns as output. In this example, we have a *score.py* file.
- An environment in which your model runs. The environment can be a Docker image with Conda dependencies or a Dockerfile.
- Settings to specify the instance type and scaling capacity.

The following table describes the key attributes of a deployment:

| Attribute      | Description                                                                                                                                                                                                                                                                                                                                                                                    |
|-----------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Name           | The name of the deployment.                                                                                                                                                                                                                                                                                                                                                                    |
| Endpoint name  | The name of the endpoint to create the deployment under.                                                                                                                                                                                                                                                                                                                                       |
| Model          | The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.                                                                                                                                                                                                                                    |
| Code path      | The path to the directory on the local development environment that contains all the Python source code for scoring the model. You can use nested directories and packages.                                                                                                                                                                                                                    |
| Scoring script | The relative path to the scoring file in the source code directory. This Python code must have an `init()` function and a `run()` function. The `init()` function will be called after the model is created or updated (you can use it to cache the model in memory, for example). The `run()` function is called at every invocation of the endpoint to do the actual scoring and prediction. |
| Environment    | The environment to host the model and code. This value can be either a reference to an existing versioned environment in the workspace or an inline environment specification.                                                                                                                                                                                                                 |
| Instance type  | The VM size to use for the deployment. For the list of supported sizes, see [Managed online endpoints SKU list](reference-managed-online-endpoints-vm-sku-list.md).                                                                                                                                                                                                                            |
| Instance count | The number of instances to use for the deployment. Base the value on the workload you expect. For high availability, we recommend that you set the value to at least `3`. We reserve an extra 20% for performing upgrades. For more information, see [managed online endpoint quotas](how-to-manage-quotas.md#azure-machine-learning-managed-online-endpoints).                                |

In [9]:
model = Model(path="model-1/model/sklearn_regression_model.pkl")
env = Environment(
    conda_file="model-1/environment/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="model-1/onlinescoring", scoring_script="score.py"
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

# 3. Create local endpoint and deployment

## 3.1 Create local endpoint

The goal of a local endpoint deployment is to validate and debug your code and configuration before you deploy to Azure. Local deployment has the following limitations:
* Local endpoints *do not support* traffic rules, authentication, or probe settings.
* Local endpoints support only one deployment per endpoint.
* They support local model files only. If you want to test registered models, first download them, then use `path` in the deployment definition to refer to the parent folder.

In [10]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Updating local endpoint (endpt-05240930390654) .Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'endpt-05240930390654', 'description': 'this is a sample online endpoint', 'tags': {'foo': 'bar'}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/Users/ben.dixon/.azureml/inferencing/endpt-05240930390654'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x118f2d610>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

## 3.2 Create local deployment

Now, create a deployment named `blue` under the endpoint.

In [11]:
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Creating local deployment (endpt-05240930390654 / blue) .
Building Docker image from Dockerfile
Step 1/6 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest
... ---> 948ed3518cc2
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> [Warning] The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested
 ---> Running in 8cedf404cce1
 ---> 1bdcce09aeba
Step 3/6 : WORKDIR /var/azureml-app/
 ---> [Warning] The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested
 ---> Running in 751a364096ea
 ---> 950e75f06919
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> 5490974d81d2
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> [Warning] The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested
 ---> Running in 658365c0

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpt-05240930390654', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/Users/ben.dixon/Repos/azure-experiments/notebooks'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x118f2d4f0>, 'model': Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': '7713d7a5680d37a33a7ac52530aec294', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/Users/ben.dixon/Repos/azure-experiments/notebooks'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x118f3f0d0>, 'version': '1', 'latest_versio

The `local=True` flag directs the SDK to deploy the endpoint in the Docker environment.

# 4. Verify the local deployment succeeded

## 4.1 Check the status to see whether the model was deployed without error

In [12]:
ml_client.online_endpoints.get(name=endpoint_name, local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:55002/score', 'openapi_uri': None, 'name': 'endpt-05240930390654', 'description': 'this is a sample online endpoint', 'tags': {'foo': 'bar'}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/Users/ben.dixon/Repos/azure-experiments/notebooks'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x118f70100>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

## 4.2 Get logs

In [13]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, local=True, lines=50
)

"2024-05-24T08:38:36,362697764+00:00 | gunicorn/run | \r\n2024-05-24T08:38:37,327502583+00:00 | gunicorn/run | \r\n2024-05-24T08:38:37,345203667+00:00 | gunicorn/run | ###############################################\r\n2024-05-24T08:38:37,361655417+00:00 | gunicorn/run | AzureML Inference Server\r\n2024-05-24T08:38:37,383157333+00:00 | gunicorn/run | ###############################################\r\n2024-05-24T08:38:37,403171875+00:00 | gunicorn/run | \r\n2024-05-24T08:38:37,421930875+00:00 | gunicorn/run | Starting AzureML Inference Server HTTP.\r\n2024-05-24 08:38:37,752 I [21] azmlinfsrv - Loaded logging config from /opt/miniconda/envs/inf-conda-env/lib/python3.9/site-packages/azureml_inference_server_http/logging.json\r\n\r\nAzure ML Inferencing HTTP server v0.8.4.1\r\n\r\n\r\nServer Settings\r\n---------------\r\nEntry Script Name: /var/azureml-app/onlinescoring/score.py\r\nModel Directory: /var/azureml-app/azureml-models//7713d7a5680d37a33a7ac52530aec294/1\r\nConfig File: None\r

## 4.3 Invoke the local endpoint
Invoke the endpoint to score the model by using the convenience command invoke and passing query parameters that are stored in a JSON file

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="model-1/sample-request.json",
    local=True,
)

In [ ]:
# Testing xgboost endpoint

In [21]:
ml_client.online_endpoints.invoke(
    endpoint_name="xgboost-model-endpoint-13cb9b54",
    request_file="model-1/xgboost-request.json",
    local=True,
)

'[1, 0, 2, 1]'

# 5. Deploy your online endpoint to Azure
Next, deploy your online endpoint to Azure.

## 5.1 Create the endpoint
Using the `endpoint` we defined earlier and the `MLClient` created earlier, we'll now create the endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [18]:
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpt-05240930390654.ukwest.inference.ml.azure.com/score', 'openapi_uri': 'https://endpt-05240930390654.ukwest.inference.ml.azure.com/swagger.json', 'name': 'endpt-05240930390654', 'description': 'this is a sample online endpoint', 'tags': {'foo': 'bar'}, 'properties': {'createdBy': 'Ben Dixon', 'createdAt': '2024-05-24T08:40:12.837223+0000', 'lastModifiedAt': '2024-05-24T08:40:12.837223+0000', 'azureml.onlineendpointid': '/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourcegroups/aiplatform-1/providers/microsoft.machinelearningservices/workspaces/ai-mlw-dev-01/onlineendpoints/endpt-05240930390654', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/providers/Microsoft.MachineLearningServices/locations/ukwest/mfeOperationsStatus/oe:3a3fd522-5898-4524-9e4b-7f1074ac8ae4:e12410ad-b98c-415d-a403-b15663069a38?ap

## 5.2 Create the deployment

Using the `blue_deployment` that we defined earlier and the `MLClient` we created earlier, we'll now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

# 6. Test the endpoint with sample data
Using the `MLClient` created earlier, we will get a handle to the endpoint. The endpoint can be invoked using the `invoke` command with the following parameters:
- `endpoint_name` - Name of the endpoint
- `request_file` - File with request data
- `deployment_name` - Name of the specific deployment to test in an endpoint

We will send a sample request using a [json](./model-1/sample-request.json) file. 

In [ ]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="../model-1/sample-request.json",
)

# 7. Managing endpoints and deployments

## 7.1 Get details of the endpoint

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

## 7.2 Get the logs for the new deployment
Get the logs for the green deployment and verify as needed

In [ ]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, lines=50
)

# 8. Delete the endpoint


In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)